In [7]:
# !pip install fosforml

In [5]:
#Snowpark lib
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

#ConfigParser to read ini file
import configparser

import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import DBSCAN
import pickle

from fosforml import *
from fosforml import scoring_func
from fosforml.constants import MLModelFlavours
import requests

np.random.seed(0)

config = configparser.ConfigParser()
config.read("/notebooks/notebooks/credentials.ini")

connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [2]:
with open('churn_model.pkl','rb') as f:  ## use rb while reading the fie
    churn_model = pickle.load(f)

In [43]:
@scoring_func
def score(model, request):
    payload_dict = eval(request.json["payload"])
    data = pd.DataFrame.from_dict(payload_dict)
    frame = data[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED']].copy()
    result = model.predict(frame)
    prediction = pd.DataFrame({"CHURN_PREDICTION":result})
    prediction["CHURN_PREDICTION"] = prediction["CHURN_PREDICTION"].apply(lambda x: "CHURN" if x == 1 else "NOT CHURN")
    probability = model.predict_proba(frame)
    prediction["Probability"] = [round(k[1],4) for k in probability]
    prediction = str(prediction.to_dict())
    return prediction

In [7]:
df = session.table("MEMBER_FUNDS_ENRICHED_DETAILS").to_pandas()
df = df.drop(["MEMBER_DOB_DT", 'ALLOCATION_DT', "CHURN_DT", "RETIREMENT_DT"], axis =1 )

In [44]:
payload = df.head(10).to_dict()
req = requests.Request()
req.json = {"payload":str(payload)}
y = req
yo = score(churn_model, y)
yo

"{'CHURN_PREDICTION': {0: 'NOT CHURN', 1: 'NOT CHURN', 2: 'CHURN', 3: 'NOT CHURN', 4: 'NOT CHURN', 5: 'NOT CHURN', 6: 'NOT CHURN', 7: 'NOT CHURN', 8: 'NOT CHURN', 9: 'CHURN'}, 'Probability': {0: 0.3505, 1: 0.3511, 2: 0.5162, 3: 0.2224, 4: 0.2325, 5: 0.2897, 6: 0.3505, 7: 0.2668, 8: 0.2126, 9: 0.5162}}"

In [41]:
frame = df[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED','CHURN_FLAG']].copy()

X = frame.drop("CHURN_FLAG", axis=1)
y = frame["CHURN_FLAG"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [42]:
payload = X_test.to_dict()
req = requests.Request()
req.json = {"payload":str(payload)}
y = req
yo = score(churn_model, y)
yo

KeyError: "['CHURN_FLAG'] not in index"

In [40]:
## registering the model in Fosfor.
model_reg = register_model(churn_model,
               score, 
               name="CHURN_BINARY_MODEL", 
               description="PREDICTING_FUND_CHURN",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install fosforml \\n pip install fosforio[snowflake] \\n pip install sklearn\\n pip install snowflake-connector-python[pandas]",
               y_true=y_test,
               y_pred=y_pred,
               prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

"{'CHURN_PREDICTION': {0: 'NOT CHURN', 1: 'NOT CHURN', 2: 'CHURN', 3: 'NOT CHURN', 4: 'NOT CHURN', 5: 'NOT CHURN', 6: 'NOT CHURN', 7: 'NOT CHURN', 8: 'NOT CHURN', 9: 'CHURN'}, 'Probability': {0: 0.3505, 1: 0.3511, 2: 0.5162, 3: 0.2224, 4: 0.2325, 5: 0.2897, 6: 0.3505, 7: 0.2668, 8: 0.2126, 9: 0.5162}}"